In [6]:
# goal is to build character level language model

In [1]:
# Download the shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
import requests
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
response = requests.get(url)
if response.status_code == 200:
    with open('./input.txt', 'wb') as f:
        f.write(response.content)
        print('file downloaded successfully')
else:
    print('file download failed')

file downloaded successfully


In [2]:
# read it and inspect it
with open('./input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
print('length of dataset in characters: ', len(text))

length of dataset in characters:  1115394


In [3]:
type(text)

str

In [4]:
# lets look at first 1000 lines
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size)
print(''.join(chars))

65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [21]:
# tokenize/encoding
# one of many possible encodings

# subword encodings
# google uses sentence piece -> https://github.com/google/sentencepiece
# openai uses tiktoken -> https://github.com/openai/tiktoken

In [22]:
# character level encodings

# create a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}    # string to integer mapping
itos = {i:ch for i, ch in enumerate(chars)}    # integer to string mapping

encode = lambda s: [stoi[c] for c in s]   # take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l])    # take a list of integers, output a string

print(encode('hi there'))
print(decode(encode('hi there')))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [23]:
# import tiktoken
# enc = tiktoken.get_encoding('gpt2')
# print(enc.n_vocab)    # number of tokens
# print(enc.encode('hii there'))
# print(enc.decode(enc.encode('hi there')))

# '''
# 50257
# [71, 4178, 612]
# hi there
# '''

In [25]:
# tokenize the entire text dataset and store it into torch.Tensor

import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])    # the 1000 characters we looked earlier will look to gpt like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [28]:
# split data into train and validation sets
n = int(0.9 * len(data))     # first 90% will be training
train_data = data[:n]
val_data = data[n:]

In [31]:
# dataloaders
block_size = 8    # context length, model sees context from length = {1, block_size}
train_data[:block_size + 1]

'''
in context of 18, 47 comes next
in context of 18 -> 47, 56 comes next
in context of 18 -> 47 -> 56, 57 comes next
.
.
.
in context of 18 -> 47 -> 56 -> 57 -> 58 -> 1 -> 15 -> 47, 58 comes next

a total of 8 examples

block_size is the time dimension of the network
'''

'\nin context of 18, 47 comes next\nin context of 18 -> 47, 56 comes next\nin context of 18 -> 47 -> 56, 57 comes next\n.\n.\n.\nin context of 18 -> 47 -> 56 -> 57 -> 58 -> 1 -> 15 -> 47, 58 comes next\n\na total of 8 examples\n\nblock_size is the time dimension of the network\n'

In [32]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f'when input is {context} the target: {target}')

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [ ]:
torch.manual_seed(1337)
